![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import re
from datetime import datetime, timedelta 
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")
import pycountry
import requests
import json
pd.options.mode.chained_assignment = None

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [4]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [5]:
df_train.info()

In [6]:
df_train.head(5)

In [7]:
df_test.info()

In [8]:
df_test.head(5)

In [9]:
sample_submission.head(5)

In [10]:
sample_submission.info()

In [11]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [12]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [13]:
data.sample(5)

In [14]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

In [15]:
for i in list(data.columns):
    print(i, len(data[i].unique()))
# Так как уникальных значений ID_TA и URL_TA меньше 50,000, данные могут содержать дубликаты

In [16]:
#Создание DF содежащего дубликаты по признаку ID_TA
duplicates=data[data.ID_TA.duplicated() == True].sort_values(by ='ID_TA')
duplicates.head()

In [17]:
duplicates.City.unique()
# Судя по всему, данные задвоеннны по City == Madrid|City == Warsaw

In [18]:
difference_check = []
for i in list(duplicates.ID_TA):
    # Ниже вычисление разницы в целевой переменной для дубликатов
    dif = list(data[data.ID_TA == i].Rating)[0] - list(data[data.ID_TA == i].Rating)[1]
    difference_check.append(dif)
difference_check

In [19]:
duplicates.Rating.value_counts()

In [20]:
# Разница в целевой переменной между дублями варируется в том же диапазоне, что и переменная в срезе
data[data.ID_TA == duplicates.ID_TA.iloc[0]]

In [21]:
data[data.ID_TA == duplicates.ID_TA.iloc[1]]
# Судя по всему, разница между некоторыми дублями объясняется тем, что они распределены между файлами kaggle_task и main_task.
# Имеет смысл создать переменную, которая обозначит, что у строки есть дубль


In [22]:
# Помечаем новыми пизнаками превые и последние дубликаты
data['1'] = data.ID_TA.duplicated(keep='first')
data['2'] = data.ID_TA.duplicated(keep='last')

In [23]:
# Функция обрабатывает всмогательные столюцы и возвращает 1 для строк имеющих дубли
def duplicate(row):
    result = 0
    if row['1'] == True:
        result = 1
    elif row['2'] == True:
        result = 1
    return result

data['has_duplicate'] = data.apply(duplicate, axis = 1)
# Удаление вспомогательных столбцов
data.drop(['1','2'], axis = 'columns', inplace = True)

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

In [24]:
# Анализ, преобразование и заполнение признаков
# Restaurant_id
# Предположительные сети ресторанов
# Создание серии содержащей ID и кол-во ресторанов
id_vc = data.groupby(['Restaurant_id']).Ranking.count()
id_vc

In [25]:
# Добавление полученной информации в DF
data['id_amount'] = data.Restaurant_id.apply(lambda x: id_vc[x])

In [26]:
sns.scatterplot(x="id_amount", y="Rating",
                sizes=(1, 8), linewidth=0,
                data=data)
# Для крупных сетей ниже разброс ненулевых значений

In [27]:
# Price Range
data['Price Range'].unique()

In [28]:
# 3. Преобразование и заполнение пропусков
# Пометка данных имеющих пропуски
data['price_class_nan'] = pd.isna(data['Price Range']).astype('uint8')
#Создание словоря для преобразования класса цены в число, где 1 - самые дешевые рестораны.
mapping_dict = {data['Price Range'].unique()[0]: 2,
                data['Price Range'].unique()[2]: 3,
                data['Price Range'].unique()[3]: 1}
# Преобразование признака в числовой формат для работы с пропусками
data['price_class'] = data['Price Range'].map(mapping_dict)
# Функция вычисления среднего класса по городу и квинтилю Ranking для города
def main_price(city, i):
    data_c = data[data.City == city]
    result = data_c[(data_c.Ranking.quantile(i) <= data_c.Ranking) & (data_c.Ranking < data_c.Ranking.quantile(i+0.2))].price_class.dropna().mean()
    return result
# Функция вычисляющая скрию значений по городам для определенного квинтиля
def quantile(quantile):
    series=[]
    for i in data.City.unique():
        series.append(main_price(i, quantile))
    return series
# Создание DF c итоговыми значениями
city_price = pd.DataFrame({'city': data.City.unique()})
quantiles = [x/10 for x in range(8, -1, -2)]
for i in quantiles:
    city_price[i]=quantile(i)
city_price.head()

In [29]:
#Создание списка содержащего вложенный список [город-квинтиль],
#где преоблядают дешевые рестораны.
class1_price=[]
for i in quantiles:
    for j in city_price[city_price[i]<1.5].city:
        class1_price.append([j, i])
class1_price

In [30]:
#Заполнение соответствующих значений
class1_indexes=[]
if len(class1_price)>0:
    #Цикл достает индексы строк соответствующих парам [город-квинтиль]
    for pair in class1_price:
        city = pair[0]
        Q = pair[1]
        df = data[(data.City == city) & (data[data.City == city].Ranking.quantile(Q) <= data.Ranking) & (data[data.City == city].Ranking < data.Ranking.quantile(Q+0.2))]
        #Цикл оставляет только пустые ячейки
        for i in df[df.price_class.isna()].index:
            class1_indexes.append(i)
    #Цикл заполния пустых значений
    for i in class1_indexes:
        data.xs(i)['price_class'] = 1 

In [31]:
#Создание списка содержащего вложенный список [город-квинтиль], где преоблядают дорогие рестораны.
#Не сотря на то, что список пуст, он может быть актуален для других данных
class3_price=[]
for i in quantiles:
    for j in city_price[city_price[i]>=2.5].city:
        class1_price.append([j, i])
class3_price

In [32]:
#Заполнение соответствующих значений аналогично дешевым ресторанам
class3_indexes=[]
if len(class3_price)>0:
    for pair in class3_price:
        city = pair[0]
        Q = pair[1]
        df = data[(data.City == city) & (data[data.City == city].Ranking.quantile(Q) <= data.Ranking) & (data[data.City == city].Ranking < data.Ranking.quantile(Q+0.2))]
        for i in df[df.price_class.isna()].index:
            class3_indexes.append(i)
    for i in class3_indexes:
        data.xs(i)['price_class'] = 1 

In [33]:
#Заполнение оставшихся ячеек самым популярным значением
data.price_class.fillna(value = 2, inplace =True)
data.sample(5)

In [34]:
data['price_class'].hist()

In [35]:
sns.catplot(x='price_class', y='Rating',
            kind="violin", bw=.1, cut=0,
            data=data)

In [36]:
# Cuisine Style
# 4. Преобразование и заполнение пропусков
# Пометка данных имеющих пропуски
data['cuisine_nan'] = pd.isna(data['Cuisine Style']).astype('uint8')
#Создание списка из строки
data['Cuisine Style'] = data['Cuisine Style'].str.findall(r'\w+\s*\w*\s*\w*\s*\w*\s*\w*')
#Функция вычисляющая самую популярную кухню в городе.
#Значение "European" опущено, т.к. говорит о географическом происхождении не достаточно точно
def popular_cuisine(city):
    popular_values = pd.Series(data[data.City == city]['Cuisine Style'].dropna().sum()).value_counts()
    if popular_values.index[0] != 'European':
        result = popular_values.index[0]
    else:
        result = popular_values.index[1]
    return result
#Создание вспомогательного DF для более быстрого заполнения пропусков
city_cuisine = pd.DataFrame({'city': data.City.unique()})
city_cuisine['cuisine'] = city_cuisine.city.apply(popular_cuisine)
city_cuisine.head()

In [37]:
#Функция заполнения пропусков популярным значением для города
def associated_cuisine(city):
    result = city_cuisine[city_cuisine.city == city].iloc[0][1]
    return result
#Функция преобразования заполненных строк в список
def making_list(cell):
    if type(cell) == list:
        result = cell
    elif type(cell) == str:
        result = re.findall(r'\w+\s*\w*\s*\w*\s*\w*\s*\w*', cell)
    return result
#Применение вышеописанных функций + создание столбца кол-ва кухонь
data['Cuisine Style'].fillna(value=data.City.apply(associated_cuisine), inplace=True)
data['Cuisine Style'] = data['Cuisine Style'].apply(making_list)
data['cuisine_amount'] = data['Cuisine Style'].apply(lambda x: len(x))

In [38]:
# Reviews
# 5. Преобразование и создание доп. парметров
# Пропусков всего 2, можно заполнить самым популярным(пустым списом) значением
data.Reviews.fillna(data.Reviews.value_counts().index[0], inplace = True)
# Функция преобразоватия строковой даты в формат datetime
def date_conversion(cell):
    resulting_list=[]
    for i in cell:
        if int(i[:i.find('/')])<=12:
            converted_time = datetime.strptime(i, '%m/%d/%Y')
            resulting_list.append(converted_time)
        else:
            if '/' in i[-4:]:
                converted_time = datetime.strptime(i, '%d/%m/%y')
                resulting_list.append(converted_time)
            else:
                converted_time = datetime.strptime(i, '%d/%m/%Y')
                resulting_list.append(converted_time)
    return resulting_list
# Создание параметра review_dates содержащего список дат отзывов
data['review_dates'] = data.Reviews.str.findall(r'\d+/\d+/\d+')
data.review_dates = data.review_dates.dropna().apply(date_conversion)
# Фунция вычисляет разницу между превым и последним отзывом
def review_t_dif(cell):
    if len(cell)>=2:
        dif=max(cell)-min(cell)
    else:
        dif=timedelta(days = 0)
    return dif
# Параметр review_time_span отбражает разницу между превым и последним отзывом
data['review_time_span'] = data.review_dates.dropna().apply(review_t_dif)
#Перевод результата в секунды
data.review_time_span = data.review_time_span.apply(lambda x: x.total_seconds())
# Параметр visible_reviews отбражает кол-во видимых отзывов
data['visible_reviews'] = data.review_dates.apply(lambda x: len(x))
data.sample(5)

In [39]:
sns.scatterplot(x='review_time_span', y='Rating',
                sizes=(1, 8), linewidth=0,
                data=data)

In [40]:
data['visible_reviews'].hist()

In [41]:
sns.catplot(x='visible_reviews', y='Rating',
            kind="violin", bw=.1, cut=0,
            data=data)

In [42]:
# 6. Применение списков ключевых слов для данных
# Параметр review_wordbox включает список слов используемых в Reviews
data.Reviews = data.Reviews.apply(lambda x: x.lower())
data['review_wordbox'] = data.Reviews.str.findall(r'\w[a-z]+')
# Все слова, встречающиеся более 2 раз у ресторанов с целевой переменной от 4.5
wordbox_positive = pd.Series(data[(data['sample'] == 1)&(data.Rating>=4.5)].review_wordbox.sum())
p_words = pd.DataFrame(wordbox_positive.value_counts())
p_words = p_words[p_words[0]>2]
# Все слова, встречающиеся более 2 раз у ресторанов с целевой переменной до 3.5
wordbox_negative = pd.Series(data[(data['sample'] == 1)&(data.Rating<=3.5)].review_wordbox.sum())
n_words = pd.DataFrame(wordbox_negative.value_counts())
n_words = n_words[n_words[0]>2]
# Слова, уникальные для ресторанов с высоким рейтингом
positive_review_predictors = []
for i in p_words.index:
    if i not in list(n_words.index):
        positive_review_predictors.append(i)
# Слова, уникальные для ресторанов с низким рейтингом
negative_review_predictors = []
for i in n_words.index:
    if i not in list(p_words.index):
        negative_review_predictors.append(i)
# Функция расчитывающая рейтинг по ключевым словам. Нейтральное значение = 0.5
def valued_review_score(wordbox):
    positives = 1
    negatives = 1
    for i in wordbox:
        if i in positive_review_predictors:
            positives += 1
        elif i in negative_review_predictors:
            negatives += 1
    result = positives/(positives + negatives)
    return result
# Параметр valued_review содержит условный рейтинг, расчитаный на осонове слов, содежащихся в Reviews
data['valued_review'] = data.review_wordbox.apply(valued_review_score)
sns.scatterplot(x='valued_review', y='Rating',
                sizes=(1, 8), linewidth=0,
                data=data)

In [43]:
# Number of Reviews
# 7. Заполнение пропусков
# Пометка данных имеющих пропуски
data['NoF_nan'] = pd.isna(data['Number of Reviews']).astype('uint8')
# Функция вычисления среднего класса по городу и квинтилю Ranking для города
def avg_reviews(city, i):
    data_c = data[data.City == city]
    result = data_c[(data_c.Ranking.quantile(i) <= data_c.Ranking) & (data_c.Ranking < data_c.Ranking.quantile(i+0.2))]['Number of Reviews'].dropna().mean()
    return result
# Функция вычисляющая серию значений по городам для определенного квинтиля
def quantile(quantile):
    series=[]
    for i in data.City.unique():
        series.append(avg_reviews(i, quantile))
    return series
# Создание DF c итоговыми значениями
city_rev = pd.DataFrame({'city': data.City.unique()})
quantiles = [x/10 for x in range(8, -1, -2)]
for i in quantiles:
    city_rev[i]=quantile(i)
city_rev.head()

In [44]:
# Функция возвращает соответсвующее значение из city_rev по City и Ranking
def aprx_rev_amount(x):
    d = data[data.City == x['City']]
    quantiles = [n/10 for n in range(8, -1, -2)]
    for i in quantiles:
        if d.Ranking.quantile(i) <= x['Ranking'] <= d.Ranking.quantile(i+0.2):
            result = float(int(city_rev[city_rev.city == x['City']][i]))
    return result
# Создание серии, содержащей индексы пустых значений с соответствующее значение из city_rev
# fillna() не ипользовал, т.к. рачеты занимали больше времени
reviews_for_nan = data[data['Number of Reviews'].isna() == True].apply(aprx_rev_amount, axis=1)
# Заполнение пропусков в основном DF
for i in list(reviews_for_nan.index):
    data['Number of Reviews'].loc[i] = reviews_for_nan[i]

In [45]:
# Ranking
# 8. Нормализация в зависимости от City
# Создание словаря, содержащего пары город: длина вектора Ranking по городу
v_lengths = {}
for i in list(data.City.unique()):
    v_lengths[i] = np.linalg.norm(data[data.City == i].Ranking)
# Функция возвращает деление значения Ranking и длины вектора Ranking по городу
def normalization(row):
    result = row.Ranking/v_lengths[row.City]
    return result
# Параметр ranking_norm содержит нормализованный Ranking
data['ranking_norm'] = data.apply(normalization, axis = 1)
sns.scatterplot(x='ranking_norm', y='Rating',
                sizes=(1, 8), linewidth=0,
                data=data)

In [46]:
# City

# 9. Создание доп. параметров на основе City
#Создание списка уникальных значений City
mdf = pd.read_csv('/kaggle/input/geonamesallcitieswithapopulation1000csv/geonames-all-cities-with-a-population-1000.csv', sep=';')
city_state = pd.DataFrame({'city': data.City.unique()})
# Функция возвращает код страны по названию города
def country(city):
    coordinates = geolocator.geocode(city)[1]
    location = geolocator.reverse(coordinates, exactly_one=True)
    address = location.raw['address']
    country = address.get('country_code', '')
    return country
# Функция возвращает население по названию города и коду страны 
# def city_population(row):
#     city = row.city
#     country = row.country
#     tmp = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&q=%s&sort=population&facet=country&refine.country=%s'
#     cmd = tmp % (city, country)
#     print(city, cmd)
# #     res = requests.get(cmd)
#     dct = json.loads(res.content)
#     info = dct['records'][0]['fields']
#     if 'population' in info.keys():
#         result = info['population']
#     else:
#         result = 0
#     return result
def city_population(row):
    city = row.city
    country = str(row.country)
    res = mdf[(mdf.Name == city) & (mdf['Country Code'] == country.upper())].Population.values
    res = res[0] if len(res) > 0 else 0
    return res

# Применение функций, перевод кода страны в верхний регистр, замена на название страны по коду
city_state['country'] = city_state.city.apply(country)
city_state['population'] = city_state.apply(city_population, axis = 1)
city_state.country = city_state.country.apply(lambda x: x.upper())
city_state.country = city_state.country.apply(lambda x: pycountry.countries.get(alpha_2=x).name)
# Список с GIT, содержащий пары страна, прилагательное
demonyms = pd.read_csv('/kaggle/input/country-adjective-pairs/demonyms.csv')
# Функция возвращает список прилагательных соответствующих стране
# Если функция возващает пустой список, сокращенное название страны меняется на официальное
def adjectives(country):
    result = list(demonyms[demonyms.Aalborg == country].Aalborgenser)
    if len(result) == 0:
        alternative = pycountry.countries.get(name=country).official_name
        result = list(demonyms[demonyms.Aalborg == alternative].Aalborgenser)
    return result
# Применение функции
city_state['adjectives'] = city_state.country.apply(adjectives)
city_state.head()

In [47]:
# Функция заполняет параметр native_cuisine.
# ==1, если один из тегов Cuisine Style совпадает с прилагательными города
def native(row):
    result = 0
    for i in city_state[city_state.city == row.City].adjectives.sum():
        if i in row['Cuisine Style']:
            result = 1
    return result
# Параметр native_cuisine отбражает евляется ли кухня национальной
data['native_cuisine'] = data.apply(native, axis = 1)
#Создание словоря с парами город - население
mapping_population = {}
for i in city_state.index:
    mapping_population[city_state.city[i]]=city_state.population[i]
# Добавление населения
# Создания параметра отнашения населения и ранка ресторана.
# Таким образом, наибольшим значением обладают высокоранговые рестораны в больших городах
data['city_population'] = data.City.map(mapping_population)
data['feature_1']=data['city_population']/data['Ranking']
data.sample(5)

In [48]:
data['native_cuisine'].hist(bins=10)

In [49]:
sns.catplot(x='native_cuisine', y='Rating',
            kind="violin", bw=.1, cut=0,
            data=data)

In [50]:
data['city_population'].hist(bins=10)

In [51]:
sns.scatterplot(x='feature_1', y='Rating',
                sizes=(1, 8), linewidth=0,
                data=data)

In [52]:
# Промежуточный итог
data.info()

In [53]:
data.describe()

In [54]:
correlation = data.corr()
ax = plt.subplots(figsize=(15, 15))
sns.heatmap(correlation, annot=True, cmap='vlag', linewidths=1, center=0)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Для примера я возьму столбец Number of Reviews
# data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
# data['Number_of_Reviews_isNAN']

In [ ]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
# data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
# data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
# data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [ ]:
# data.head(5)

In [55]:
# 10. Dummy-variables
# 10.1 City
# Создаем сетку
data = pd.get_dummies(data, columns=[ 'City'], dummy_na=False)
data.sample(5)

In [60]:
# 10.2 Cuisine Style
# Функция для заполнения категории Cuisine Style
def dummy_cuisine(cell):
    if item in cell:
        return 1
    return 0
# Список уникальных значений переменной
all_cuisines = pd.Series(data['Cuisine Style'].sum()).unique()
# Применение функции
for item in all_cuisines:
    data[item] = data['Cuisine Style'].apply(dummy_cuisine)
data.sample(3)

In [61]:
# 10.3 Price Range
# Создаем сетку
data = pd.get_dummies(data, columns=[ 'price_class'], dummy_na=False)
data.sample(3)

In [62]:
# Предобработка
# Очистка от строк/списков
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
object_columns

In [63]:
data.drop(object_columns, axis = 1, inplace=True)
df_preproc = data
df_preproc.sample(5)

In [64]:
df_preproc.info()

#### Возьмем следующий признак "Price Range".

In [ ]:
# data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
# Ваша обработка 'Price Range'

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [ ]:
# тут ваш код на обработку других признаков
# .....

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
# plt.rcParams['figure.figsize'] = (10,7)
# df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
# df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
# df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
# for x in (df_train['City'].value_counts())[0:10].index:
#     df_train['Ranking'][df_train['City'] == x].hist(bins=100)
# plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
# df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
# df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
# df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
# plt.rcParams['figure.figsize'] = (15,10)
# sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
# df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
# df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
# df_train['sample'] = 1 # помечаем где у нас трейн
# df_test['sample'] = 0 # помечаем где у нас тест
# df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

# data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
# data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # ....
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
# df_preproc = preproc_data(data)
# df_preproc.sample(10)

In [ ]:
# df_preproc.info()

In [65]:
# Разделение данных
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [66]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [67]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [68]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [69]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [70]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [71]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [72]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [73]:
test_data.sample(10)

In [74]:
test_data = test_data.drop(['Rating'], axis=1)

In [75]:
sample_submission

In [76]:
predict_submission = model.predict(test_data)

In [77]:
predict_submission

In [79]:
adjusted_submission=[]
for i in predict_submission:
    if i%1>=0.75:
        adjusted_submission.append(int(i)+1)
    elif 0.75>i%1>=0.25:
        adjusted_submission.append(int(i)+0.5)
    else:
        adjusted_submission.append(int(i))
adjusted_submission[:5]

In [80]:
sample_submission['Rating'] = adjusted_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
